### Imports & Definitions

In [2]:
import sys
sys.path.append("../../utils")
from definitions import *
# from path_helpers import get_dataset_path, get_metric_dir_path
from mera_helpers import construct_prompt
from huggingface_helpers import get_tokenizer, get_model
from llm_helpers import calculate_token_interest_probs, get_answer

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
 

### Learn to load asessors dataset from disk

In [1]:
! pwd

/home/jupyter/work/resources/long_context_LLMs/src/notebooks/metrics_calculation


In [3]:
df = pd.read_csv("/home/jupyter/mnt/datasets/spbu_diplomas/russian_dataset/russian_dataset_test.csv")
df.head()

,id,year,diploma,abstract,study_field,degree,original_diploma_extension
0,41453,2023,Санкт-Петербургский государственный университе...,Абдуллаев Ш.У. тема диссертации: «Роль политик...,INTERNATIONAL RELATIONS,MASTER'S STUDIES,.doc
1,43790,2023,Санкт-Петербургский государственный университе...,Выпускная квалификационная работа посвящена из...,POLITICAL SCIENCE,BACHELOR STUDIES,.docx
2,41165,2023,Санкт-Петербургский государственный университе...,В данной работе описывается реализация обобщен...,MATHEMATICS AND MECHANICS,BACHELOR STUDIES,.pdf
3,42349,2023,ПРАВИТЕЛЬСТВО РОССИЙСКОЙ ФЕДЕРАЦИИ \nСАНКТ-ПЕТ...,В выпускной квалификационной работе раскрывает...,SOCIOLOGY,BACHELOR STUDIES,.docx
4,40166,2023,Федеральное государственное бюджетное образова...,На сегодняшний день наблюдается высокий спрос ...,MANAGEMENT,BACHELOR STUDIES,.docx


In [4]:
with open(ARTIFACTS_DIR_PATH.joinpath("datasets/diplomas_asessors_questions/mcs_df_human_filled_processed.json"), "r") as f:
    dataset = json.load(f)

In [5]:
dataset[0]

{'instruction': 'Задание содержит контексти и вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\nКонтекст: {context}\nВопрос: {text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:',
 'inputs': {'text': 'Какой объект строят в этой работе?',
  'option_a': 'В этой работе ничего не строят',
  'option_b': 'Правую трансферную модельную структуру',
  'option_c': 'Моноид в категории эндофункторов',
  'option_d': 'Левую трансферную модельную структуру',
  'subject': 'Математика и компьютерные науки',
  'context': 'В этой работе мы строим правую трансферную модельную структуру на категории алгебр обогащенной алгебраической теории в обогащенной модельной категории, удовлетворяющей некоторым свойствам. Также мы даем несколько общих конструкций генерирующих замкнутые симметричные моноидальные локально конечно представимые категори, которые служат базой обогащения.'},
 'outputs': 'D',
 'meta': {'abstract': 

In [6]:
x = copy.deepcopy(dataset[0])
x['inputs']['context'] = "" # df["diploma"].iloc[int(x['meta']['id'])]
q = construct_prompt(x)
print(q)

Задание содержит контексти и вопрос по теме Математика и компьютерные науки и 4 варианта ответа A, B, C, D, из которых только один правильный.
Контекст: 
Вопрос: Какой объект строят в этой работе?
A В этой работе ничего не строят
B Правую трансферную модельную структуру
C Моноид в категории эндофункторов
D Левую трансферную модельную структуру
Запишите букву правильного ответа
Ответ:


### Learn to load datasets from disk

In [2]:
for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
    for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
        path = get_dataset_path(subset, name, split)
        dataset = load_from_disk(path)
        print(subset, dataset[0])

rummlu {'instruction': 'Задание содержит вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\n{text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:', 'inputs': {'text': 'В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.', 'option_a': 'Нарушает, Нарушает', 'option_b': 'Нарушает, Не нарушает', 'option_c': 'Не нарушает, Нарушает', 'option_d': 'Не нарушает, Не нарушает', 'subject': 'Философия'}, 'outputs': 'B', 'meta': {'domain': 'moral_scenarios', 'id': 0}}
ruopenbookqa {'instruction': '{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A

In [3]:
rummlu = {'instruction': 'Задание содержит вопрос по теме {subject} и 4 варианта ответа A, B, C, D, из которых только один правильный.\n{text}\nA {option_a}\nB {option_b}\nC {option_c}\nD {option_d}\nЗапишите букву правильного ответа\nОтвет:', 'inputs': {'text': 'В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.', 'option_a': 'Нарушает, Нарушает', 'option_b': 'Нарушает, Не нарушает', 'option_c': 'Не нарушает, Нарушает', 'option_d': 'Не нарушает, Не нарушает', 'subject': 'Философия'}, 'outputs': 'B', 'meta': {'domain': 'moral_scenarios', 'id': 0}}
construct_prompt(rummlu)

'Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.\nВ какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.\nA Нарушает, Нарушает\nB Нарушает, Не нарушает\nC Не нарушает, Нарушает\nD Не нарушает, Не нарушает\nЗапишите букву правильного ответа\nОтвет:'

In [4]:
ruopenbookqa = {'instruction': '{question}\nA. {option_a}\nB. {option_b}\nC. {option_c}\nD. {option_d}\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: ', 'inputs': {'question': 'Примером потомства, получившего генетический набор от родителей, является', 'option_a': 'маленький кротенок', 'option_b': 'маленький мешок сахара', 'option_c': 'маленькая тележка', 'option_d': 'маленький пирожок'}, 'outputs': 'A', 'meta': {'id': 0}}
construct_prompt(ruopenbookqa)

'Примером потомства, получившего генетический набор от родителей, является\nA. маленький кротенок\nB. маленький мешок сахара\nC. маленькая тележка\nD. маленький пирожок\nКакой ответ является правильным? Запишите только букву верного варианта: A, B, C или D.\nОтвет: '

### Learn to inference LLM on these datasets

In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [10]:
for baseline_model in HUGGINFACE_BASELINE_MODELS:
    tokenizer = get_tokenizer(baseline_model, HUGGINGFACE_MODEL_TO_REPO[baseline_model])
    model = get_model(baseline_model, HUGGINGFACE_MODEL_TO_REPO[baseline_model])
    model.to(device)
    q = construct_prompt(rummlu)
    print(q)
    probs = calculate_token_interest_probs(q, tokenizer, model)
    print(probs)
    a = get_answer(probs)
    print(a)

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.02s/it]


Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.
В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.
A Нарушает, Нарушает
B Нарушает, Не нарушает
C Не нарушает, Нарушает
D Не нарушает, Не нарушает
Запишите букву правильного ответа
Ответ:
{'A': 0.08026193082332611, 'B': 0.09532195329666138, 'C': 0.09801502525806427, 'D': 0.07560590654611588}
C


Loading checkpoint shards: 100%|██████████| 2/2 [02:17<00:00, 68.77s/it] 


Задание содержит вопрос по теме Философия и 4 варианта ответа A, B, C, D, из которых только один правильный.
В какой из этих двух ситуаций действующее лицо, использующее местоимения "я"/"меня"/"мой", нарушает нормы морали, общепринятые в США по состоянию на 2020 год? Ситуация 1 | Я оторвал голову детской игрушке. Ситуация 2 | Я узнал, что моей сестре нравится один мальчик, она не хотела, чтобы он знал, и я сохранил это в тайне.
A Нарушает, Нарушает
B Нарушает, Не нарушает
C Не нарушает, Нарушает
D Не нарушает, Не нарушает
Запишите букву правильного ответа
Ответ:
{'A': 0.1555231362581253, 'B': 0.18379676342010498, 'C': 0.12277740240097046, 'D': 0.12690876424312592}
B


In [11]:
%%time
calculate_token_interest_probs(q, tokenizer, model)

CPU times: user 262 ms, sys: 24.9 ms, total: 287 ms
Wall time: 285 ms


{'A': 0.1555231362581253,
 'B': 0.18379676342010498,
 'C': 0.12277740240097046,
 'D': 0.12690876424312592}

### Inference LLMs on asessors dataset without context at all and with full diploma text 

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [8]:
from huggingface_hub import notebook_login
notebook_login()

In [9]:
from tqdm.notebook import tqdm

for baseline_model in tqdm([LLAMA_2_7B, MISTRAL_7B], desc="Models..."):
    repo = HUGGINGFACE_MODEL_TO_REPO[baseline_model]
    tokenizer = get_tokenizer(baseline_model, repo)
    model = get_model(baseline_model, repo)
    model.to(device)
    probs_list = []
    a_list = []
    for row in tqdm(dataset, desc="Rows..."):
        x = copy.deepcopy(row)
        x['inputs']['context'] = "" # df["diploma"].iloc[int(x['meta']['id'])]
        q = construct_prompt(row)
        probs = calculate_token_interest_probs(q, tokenizer, model)
        probs_list.append({
            "probs": probs,
            "meta": row["meta"],
        })
        a = get_answer(probs)
        a_list.append({
            "answer": a,
            "meta": row["meta"],
        })
    metric_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/empty/{baseline_model}/") # get_metric_dir_path(baseline_model, subset, name, split)
    metric_dir_path.mkdir(exist_ok=True, parents=True)
    with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
        json.dump(probs_list, f, ensure_ascii=False, indent=2)
    with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
        json.dump(a_list, f, ensure_ascii=False, indent=2)

Models...:   0%|          | 0/2 [00:00<?, ?it/s]


Loading checkpoint shards: 100%|██████████| 2/2 [02:01<00:00, 60.52s/it]


Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

Loading checkpoint shards: 100%|██████████| 2/2 [02:13<00:00, 66.94s/it]


Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

In [11]:
from tqdm.notebook import tqdm

for baseline_model in tqdm([LLAMA_2_7B, MISTRAL_7B], desc="Models..."):
    repo = HUGGINGFACE_MODEL_TO_REPO[baseline_model]
    tokenizer = get_tokenizer(baseline_model, repo)
    model = get_model(baseline_model, repo)
    model.to(device)
    probs_list = []
    a_list = []
    for row in tqdm(dataset, desc="Rows..."):
        x = copy.deepcopy(row)
        x['inputs']['context'] = df["diploma"].iloc[int(x['meta']['id'])] # ""
        q = construct_prompt(row)
        probs = calculate_token_interest_probs(q, tokenizer, model)
        probs_list.append({
            "probs": probs,
            "meta": row["meta"],
        })
        a = get_answer(probs)
        a_list.append({
            "answer": a,
            "meta": row["meta"],
        })
    metric_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/diploma/{baseline_model}/") # get_metric_dir_path(baseline_model, subset, name, split)
    metric_dir_path.mkdir(exist_ok=True, parents=True)
    with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
        json.dump(probs_list, f, ensure_ascii=False, indent=2)
    with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
        json.dump(a_list, f, ensure_ascii=False, indent=2)

Models...:   0%|          | 0/2 [00:00<?, ?it/s]


Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.32it/s]


Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.35it/s]


Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

In [12]:
"kek"

'kek'

### Inference LLMs on asessors dataset

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [7]:
from tqdm.notebook import tqdm

for baseline_model in tqdm([LLAMA_2_7B, MISTRAL_7B], desc="Models..."):
    repo = HUGGINGFACE_MODEL_TO_REPO[baseline_model]
    tokenizer = get_tokenizer(baseline_model, repo)
    model = get_model(baseline_model, repo)
    model.to(device)
    probs_list = []
    a_list = []
    for row in tqdm(dataset, desc="Rows..."):
        q = construct_prompt(row)
        probs = calculate_token_interest_probs(q, tokenizer, model)
        probs_list.append({
            "probs": probs,
            "meta": row["meta"],
        })
        a = get_answer(probs)
        a_list.append({
            "answer": a,
            "meta": row["meta"],
        })
    metric_dir_path = ARTIFACTS_DIR_PATH.joinpath(f"metrics/diplomas_asessors_questions/abstract/{baseline_model}/") # get_metric_dir_path(baseline_model, subset, name, split)
    metric_dir_path.mkdir(exist_ok=True, parents=True)
    with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
        json.dump(probs_list, f, ensure_ascii=False, indent=2)
    with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
        json.dump(a_list, f, ensure_ascii=False, indent=2)

Models...:   0%|          | 0/2 [00:00<?, ?it/s]


Loading checkpoint shards: 100%|██████████| 2/2 [02:05<00:00, 62.54s/it]


Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.

Loading checkpoint shards: 100%|██████████| 2/2 [02:13<00:00, 66.94s/it]


Rows...:   0%|          | 0/70 [00:00<?, ?it/s]

### Inference LLMs on all datasets

In [12]:
# add tqdm
for baseline_model in tqdm(HUGGINFACE_BASELINE_MODELS, desc="Models..."):
    repo = HUGGINGFACE_MODEL_TO_REPO[baseline_model]
    tokenizer = get_tokenizer(baseline_model, repo)
    model = get_model(baseline_model, repo)
    model.to(device)
    for name, dataset_meta in tqdm(HUGGINGFACE_NAME_TO_DATASET.items(), desc="Datasets..."):
        for subset, split in tqdm(zip(dataset_meta["subsets"], dataset_meta["splits"]), desc="Splits..."):
            path = get_dataset_path(subset, name, split)
            dataset = load_from_disk(path)
            probs_list = []
            a_list = []
            for row in tqdm(dataset, desc="Rows..."):
                q = construct_prompt(row)
                probs = calculate_token_interest_probs(q, tokenizer, model)
                probs_list.append({
                    "probs": probs,
                    "meta": row["meta"],
                })
                a = get_answer(probs)
                a_list.append({
                    "answer": a,
                    "meta": row["meta"],
                })
            metric_dir_path = get_metric_dir_path(baseline_model, subset, name, split)
            metric_dir_path.mkdir(exist_ok=True, parents=True)
            with open(metric_dir_path.joinpath("probs.jsonl"), "w") as f:
                json.dump(probs_list, f, ensure_ascii=False, indent=2)
            with open(metric_dir_path.joinpath("answers.jsonl"), "w") as f:
                json.dump(a_list, f, ensure_ascii=False, indent=2)

Models...:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.06s/it]

Datasets...:   0%|          | 0/1 [00:00<?, ?it/s]

Splits...: 0it [00:00, ?it/s]


Rows...:   0%|          | 0/10033 [00:00<?, ?it/s]


Rows...:   0%|          | 1/10033 [00:00<50:49,  3.29it/s]


Rows...:   0%|          | 2/10033 [00:00<53:03,  3.15it/s]


Rows...:   0%|          | 3/10033 [00:01<57:16,  2.92it/s]


Rows...:   0%|          | 4/10033 [00:01<58:48,  2.84it/s]


Rows...:   0%|          | 5/10033 [00:01<55:53,  2.99it/s]


Rows...:   0%|          | 6/10033 [00:01<53:58,  3.10it/s]


Rows...:   0%|          | 7/10033 [00:02<47:54,  3.49it/s]


Rows...:   0%|          | 8/10033 [00:02<43:57,  3.80it/s]


Rows...:   0%|          | 9/10033 [00:02<43:01,  3.88it/s]


Rows...:   0%|          | 10/10033 [00:02<45:16,  3.69it/s]


Rows...:   0%|          | 11/10033 [00:03<42:32,  3.93it/s]


Rows...:   0%|          | 12/10033 [00:03<40:19,  4.14it/s]


Rows

In [20]:
device = torch.cuda.empty_cache()

In [19]:
! nvidia-smi

Tue Apr 30 17:07:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB           On  | 00000000:8C:00.0 Off |                  Off |
| N/A   33C    P0              36W / 250W |  31500MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [25]:
torch.cuda.empty_cache()

In [26]:
! nvidia-smi

Tue Apr 30 17:10:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.161.08             Driver Version: 535.161.08   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-PCIE-32GB           On  | 00000000:8C:00.0 Off |                  Off |
| N/A   32C    P0              36W / 250W |  31428MiB / 32768MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--